In [ ]:
# default_exp simulator

# Simulator

> Defines a unified interface for working with quantum state simulators.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from qsam.tools import patch

We always need to specify the number of qubits to specify a state. All qubits are initialized in the $|00..0\rangle$ state for all simulators.

In [ ]:
#export
class Simulator:
    """Interface for quantum state simulation"""
    def __init__(self, num_qubits):
        self.n_qubits = num_qubits

In order to run a state simulation, i.e. propagation of an initial state through a quantum circuit, we iterate over the `ticks` of a circuit and apply gates on the specified qubits. Each simulator implements a certain set of gates we can call. However, the user is not expected to call the `_apply_gate` method but to use the `run` method on entire `circuit`s instead.

In [ ]:
#export
@patch(Simulator)
def _apply_gate(self, gate_symbol, qubits):
    """Apply a gate to the `qubits` of the current state."""
    gate = getattr(self, gate_symbol.lower())    
    args = (qubits,) if type(qubits)==int else qubits
    return gate(*args)

Thus, we can only run gates which are implemented:

In [ ]:
s = Simulator(num_qubits=3)
try:
    s._apply_gate("X", 2) 
except AttributeError as e:
    print(e)

'Simulator' object has no attribute 'x'


As expected, we get an AttributeError. However, when we implement this gate in the simulator..

In [ ]:
@patch(Simulator)
def x(self,qubit):
    pass

s = Simulator(num_qubits=3)
s._apply_gate("X", 2) 

..we get no error. Now, in order to simulate the (new) state after a complete circuit application, we apply each tick sequentially in the `run` method. This method is the main interface to the simulator for a user. Additionally, the `run` method allows to pass a `fault_circuit` which is a "blank" copy of `circuit`, i.e. it has the same number of ticks and qubits, only that it has fault gates at specified positions. Fault gates are just regular gates which are applied at the end of a tick.

In [ ]:
#export
@patch(Simulator)
def run(self, circuit, fault_circuit=None):
    """Apply gates in `circuit` sequentially to current state.
    If `fault_circuit` is specified apply fault gates after each tick."""
    measurements = []
    for tick_index, tick in enumerate(circuit):
        if type(tick) == list:
            for sub_tick in tick:
                res = self._apply_gate(*sub_tick)
                if res: measurements.append( (tick_index,res) )
        elif type(tick) == tuple:
            res = self._apply_gate(*tick)
            if res: measurements.append( (tick_index,res) )
        
        if fault_circuit:
            fault_tick = fault_circuit[tick_index]
            if fault_tick:
                if type(fault_tick) == list:
                    for sub_fault_tick in fault_tick:
                        self._apply_gate(*sub_fault_tick)
                elif type(fault_tick) == tuple:
                    self._apply_gate(*fault_tick)
    return measurements

The `run` method will return a list of all measurements (if any) per tick. 